In [1]:
import numpy as np
import torch

In [2]:
#array
a = np.array(1)
b = torch.tensor(1)

print(a)
print(b)

1
tensor(1)


In [3]:
type(a),type(b)

(numpy.ndarray, torch.Tensor)

In [4]:
# setting random seed for numpy
np.random.seed(42)
# matrix for random numbers
a = np.random.randn(3,3)
a

array([[ 0.49671415, -0.1382643 ,  0.64768854],
       [ 1.52302986, -0.23415337, -0.23413696],
       [ 1.57921282,  0.76743473, -0.46947439]])

In [5]:
# setting random seed for torch
torch.manual_seed(42)
# matrix for random numbers
b = torch.randn(3,3)
b

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863],
        [ 2.2082, -0.6380,  0.4617]])

In [6]:
# creating matrices
a = torch.tensor([[1,2], [3,4]])
b = torch.tensor([[4,6], [7,8]])
print(a)
print(b)

tensor([[1, 2],
        [3, 4]])
tensor([[4, 6],
        [7, 8]])


In [7]:
# vertical concatenation
torch.cat((a,b))

tensor([[1, 2],
        [3, 4],
        [4, 6],
        [7, 8]])

In [8]:
# horizontal concatenation
torch.cat((a,b),dim=1)

tensor([[1, 2, 4, 6],
        [3, 4, 7, 8]])

In [9]:
# setting random seed
torch.manual_seed(42)
# creating matrix
a = torch.randn(2,4)
print(a)
a.shape

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303],
        [-1.1229, -0.1863,  2.2082, -0.6380]])


torch.Size([2, 4])

In [10]:
# reshaping tensor
b = a.reshape(1,8)
print(b)
b.shape

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229, -0.1863,  2.2082, -0.6380]])


torch.Size([1, 8])

# Common PyTorch Modules

## Autograd Module

PyTorch uses a technique called automatic differentiation. It records all the operations that we are performing and replays it backward to compute gradients. This technique helps us to save time on each epoch as we are calculating the gradients on the forward pass itself.

In [11]:
a = torch.ones((2,2), requires_grad=True)
a

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

Here, we have initialized a tensor. Specifying requires_grad as True will make sure that the gradients are stored for this particular tensor whenever we perform some operation on it. Let’s now perform some operations on the defined tensor:

In [12]:
b = (a + 5).mean()
b

tensor(6., grad_fn=<MeanBackward0>)

Now, the derivative of c w.r.t. a will be ¼ and hence the gradient matrix will be 0.25. Let’s verify this using PyTorch:

In [13]:
# back propagation
b.backward()
# computing gradients
print(a.grad)

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])


### The autograd module helps us to compute the gradients in the forward pass itself which saves a lot of computation time of an epoch.

## Optim Module

The Optim module in PyTorch has pre-written codes for most of the optimizers that are used while building a neural network. We just have to import them and then they can be used to build models.

In [14]:
# importing the optim module
from torch import optim

# adam
## adam = optim.Adam(model.parameters(), lr=learning_rate)

# sgd
## SGD = optim.SGD(model.parameters(), lr=learning_rate)

Above are the examples to get the ADAM and SGD optimizers. Most of the commonly used optimizers are supported in PyTorch and hence we do not have to write them from scratch.

## nn Module

The autograd module in PyTorch helps us define computation graphs as we proceed in the model. But, just using the autograd module can be low-level when we are dealing with a complex neural network.

In those cases, we can make use of the nn module. This defines a set of functions, similar to the layers of a neural network, which takes the input from the previous state and produces an output.

## Building a Neural Network from Scratch in PyTorch

In [15]:
# input
X = torch.Tensor([[1,0,1,0],[1,0,1,1],[0,1,0,1]])

# output
y = torch.Tensor([[1],[1],[0]])

print(X)
print(y)

tensor([[1., 0., 1., 0.],
        [1., 0., 1., 1.],
        [0., 1., 0., 1.]])
tensor([[1.],
        [1.],
        [0.]])


In [16]:
# sigmoid function
def sigmoid(x):
    return 1/(1 + torch.exp(-x))

# sigmoid devrivatives
def sigmoid_derivative(x):
    return x * (1 - x)

In [17]:
# variable initialization
epochs = 7000 # number of iterations
lr = 0.1 # learning rate
inputlayer_neurons = X.shape[1] # number of features
hiddenlayer_neurons = 3
output_neurons = 1

# weight and bias initialization
wh = torch.randn(inputlayer_neurons, hiddenlayer_neurons).type(torch.FloatTensor)
wout = torch.randn(hiddenlayer_neurons, output_neurons)

bh = torch.randn(1, hiddenlayer_neurons).type(torch.FloatTensor)
bout = torch.randn(1, output_neurons)

In [18]:
for i in range(epochs):
    # forward propagation
    hidden_layer_input1 = torch.mm(X, wh)
    hidden_layer_input = hidden_layer_input1 + bh
    hidden_layer_activations = sigmoid(hidden_layer_input)
    
    output_layer_input1 = torch.mm(hidden_layer_activations, wout)
    output_layer_input = output_layer_input1 + bout
    output = sigmoid(output_layer_input)
        
    # backward propagation (backpropagation)
    E = y - output
    slope_hidden_layer = sigmoid_derivative(hidden_layer_activations)
    slope_output_layer = sigmoid_derivative(output)
    d_output = E * slope_output_layer
    Error_at_hidden_layer = torch.mm(d_output, wout.t())
    d_hiddenlayer = Error_at_hidden_layer * slope_hidden_layer
    wout += torch.mm(hidden_layer_activations.t(), d_output) * lr
    bout += d_output.sum() * lr
    wh += torch.mm(X.t(), d_hiddenlayer) * lr
    bh += d_output.sum() * lr

In [19]:
print('actual:\n', y)
print('predicted:\n', output)

actual:
 tensor([[1.],
        [1.],
        [0.]])
predicted:
 tensor([[0.9869],
        [0.9775],
        [0.0316]])
